In [1]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os,glob
import numpy as np
import sklearn.preprocessing as prep
import librosa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses

In [4]:
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    #mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    mfcc = librosa.feature.melspectrogram(wave, sr=sampleRate).transpose()
    #print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

# load wav files with this format speaker_word_number.wav
class EasyASRcommands(object):
    '''
    classdocs
    '''
    def __init__(self, filePath="/media/francisco/FREEDOS/datasets/sound/speech_commands_v0.02"):
        '''
        Constructor
        '''
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.words=[]
        dComms=os.listdir(basepath)
        print("dComms=",len(dComms))
        for i,fc in enumerate(dComms):
            if(fc[0]=="_"):
                continue
            fd=basepath+"/"+fc
            #print("fd=",fd)
            if(not os.path.isdir(fd)):
                continue
            for f in os.listdir(fd):#[:1000]:
                #print(f)
                fn=fd+"/"+f
                #print("fn=",fn)
                mfccs=wav2mfcc(fn)
                self.mfccs.append(mfccs)
                speaker,_,number=f.split("_")
                self.speakers.append(speaker)
                self.words.append(fc)
            print(i,"fc=",fc,len(self.mfccs))
        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.leWords=prep.LabelEncoder()
        self.leWords.fit(self.words)
        self.iWordLabels=self.leWords.transform(self.words)
        print(len(self.mfccs))
        print(len(self.words))
        print(len(self.speakers))

ea=EasyASRcommands()
print(ea.leWords)

dComms= 41
0 fc= eight 3787
2 fc= sheila 5809
4 fc= nine 9743
5 fc= yes 13787
6 fc= one 17677
7 fc= no 21618
8 fc= left 25419
9 fc= tree 27178
10 fc= bed 29192
11 fc= bird 31256
12 fc= go 35136
13 fc= wow 37259
14 fc= seven 41257
15 fc= marvin 43357
16 fc= dog 45485
17 fc= three 49212
19 fc= two 53092
20 fc= house 55205
22 fc= backward 56869
23 fc= down 60786
24 fc= six 64646
25 fc= five 68698
26 fc= off 72443
27 fc= right 76221
28 fc= cat 78252
29 fc= zero 82304
30 fc= four 86032
32 fc= forward 87589
33 fc= stop 91461
34 fc= up 95184
35 fc= visual 96776
36 fc= learn 98351
38 fc= on 102196
39 fc= happy 104250
40 fc= follow 105829
105829
105829
105829
LabelEncoder()


In [7]:
trainX, trainYwords, trainYspeakers = np.array(ea.mfccs),np.array(ea.iWordLabels),np.array(ea.iSpeakerLabels)
#testX, testY = test

maxsamples=trainX.shape[0]
maxlen=trainX.shape[1]

shuffled_order = np.array(range(maxsamples))
np.random.shuffle(shuffled_order)

trainX         = trainX[shuffled_order]
trainYwords    = trainYwords[shuffled_order]
trainYspeakers = trainYspeakers[shuffled_order]
        
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputWords_dim=max(trainYwords)+1
outputSpeakers_dim=max(trainYspeakers)+1

print(outputWords_dim,outputSpeakers_dim)

# Converting labels to binary vectors
trainYwords_cat    = utils.to_categorical(trainYwords   ,outputWords_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
print (trainX.shape,trainYwords_cat.shape,trainYspeakers_cat.shape)

35 2618
(105829, 30, 128) (105829, 35) (105829, 2618)


In [8]:
# Network building
def build_model_asr_lstm(input_dim,outputWords_dim,outputSpeakers_dim,hiden_dim=128):
    input = Input(shape=(None,input_dim))
    #x = LSTM(hiden_dim, input_dim=input_dim, input_length=maxlen)(input)
    #x = Bidirectional(LSTM(hiden_dim))(input)
    x = Bidirectional(LSTM(hiden_dim,return_sequences=True))(input)
    #x = Dropout(0.5)(x)
    #x = Bidirectional(LSTM(hiden_dim,return_sequences=True))(x)
    x = Dropout(0.5)(x)
    x = Bidirectional(LSTM(hiden_dim))(x)
    x = Dropout(0.5)(x)
    wd= Dense(hiden_dim,activation="relu")(x)
    sd= Dense(hiden_dim,activation="relu")(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_words    = Dense(outputWords_dim   , activation="softmax", name="word")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)

    model = Model(input, [prediction_words,prediction_speakers])
    return model

In [9]:
model=build_model_asr_lstm(input_dim,outputWords_dim,outputSpeakers_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, None, 256)    263168      input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, None, 256)    0           bidirectional[0][0]              
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 256)          394240      dropout[0][0]                    
______________________________________________________________________________________________

In [ ]:
# Training
history=model.fit(trainX, [trainYwords_cat,trainYspeakers_cat], batch_size=5000,epochs=1000,validation_split=0.2)

Train on 84663 samples, validate on 21166 samples
Epoch 1/1000
84663/84663 [==============================] - 13s 148us/sample - loss: 11.0691 - word_loss: 3.3237 - speaker_loss: 7.7431 - word_accuracy: 0.1072 - speaker_accuracy: 0.0015 - val_loss: 10.3406 - val_word_loss: 2.8801 - val_speaker_loss: 7.4693 - val_word_accuracy: 0.2118 - val_speaker_accuracy: 0.0019
Epoch 2/1000
84663/84663 [==============================] - 6s 75us/sample - loss: 10.0018 - word_loss: 2.6617 - speaker_loss: 7.3387 - word_accuracy: 0.2497 - speaker_accuracy: 0.0043 - val_loss: 9.4986 - val_word_loss: 2.2930 - val_speaker_loss: 7.2153 - val_word_accuracy: 0.3385 - val_speaker_accuracy: 0.0055
Epoch 3/1000
84663/84663 [==============================] - 6s 75us/sample - loss: 9.2681 - word_loss: 2.1803 - speaker_loss: 7.0864 - word_accuracy: 0.3669 - speaker_accuracy: 0.0085 - val_loss: 8.8276 - val_word_loss: 1.8839 - val_speaker_loss: 6.9542 - val_word_accuracy: 0.4525 - val_speaker_accuracy: 0.0105
Epoch 

Epoch 27/1000
84663/84663 [==============================] - 6s 75us/sample - loss: 3.0465 - word_loss: 0.8523 - speaker_loss: 2.1944 - word_accuracy: 0.7366 - speaker_accuracy: 0.4757 - val_loss: 2.9466 - val_word_loss: 0.7604 - val_speaker_loss: 2.1999 - val_word_accuracy: 0.7737 - val_speaker_accuracy: 0.5135
Epoch 28/1000
84663/84663 [==============================] - 6s 75us/sample - loss: 2.9811 - word_loss: 0.8432 - speaker_loss: 2.1379 - word_accuracy: 0.7405 - speaker_accuracy: 0.4913 - val_loss: 2.8888 - val_word_loss: 0.7461 - val_speaker_loss: 2.1533 - val_word_accuracy: 0.7762 - val_speaker_accuracy: 0.5233
Epoch 29/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 2.9184 - word_loss: 0.8306 - speaker_loss: 2.0879 - word_accuracy: 0.7445 - speaker_accuracy: 0.4990 - val_loss: 2.8378 - val_word_loss: 0.7410 - val_speaker_loss: 2.1131 - val_word_accuracy: 0.7780 - val_speaker_accuracy: 0.5319
Epoch 30/1000
84663/84663 [==============================]

84663/84663 [==============================] - 7s 81us/sample - loss: 1.8793 - word_loss: 0.6050 - speaker_loss: 1.2746 - word_accuracy: 0.8100 - speaker_accuracy: 0.6650 - val_loss: 2.1495 - val_word_loss: 0.6193 - val_speaker_loss: 1.5556 - val_word_accuracy: 0.8173 - val_speaker_accuracy: 0.6507
Epoch 54/1000
84663/84663 [==============================] - 7s 81us/sample - loss: 1.8690 - word_loss: 0.6007 - speaker_loss: 1.2684 - word_accuracy: 0.8128 - speaker_accuracy: 0.6655 - val_loss: 2.1277 - val_word_loss: 0.6174 - val_speaker_loss: 1.5378 - val_word_accuracy: 0.8178 - val_speaker_accuracy: 0.6542
Epoch 55/1000
84663/84663 [==============================] - 6s 75us/sample - loss: 1.8366 - word_loss: 0.5951 - speaker_loss: 1.2417 - word_accuracy: 0.8132 - speaker_accuracy: 0.6704 - val_loss: 2.1109 - val_word_loss: 0.6111 - val_speaker_loss: 1.5247 - val_word_accuracy: 0.8221 - val_speaker_accuracy: 0.6566
Epoch 56/1000
84663/84663 [==============================] - 6s 76us/sam

84663/84663 [==============================] - 6s 76us/sample - loss: 1.3739 - word_loss: 0.4779 - speaker_loss: 0.8961 - word_accuracy: 0.8502 - speaker_accuracy: 0.7497 - val_loss: 1.9074 - val_word_loss: 0.5831 - val_speaker_loss: 1.3506 - val_word_accuracy: 0.8338 - val_speaker_accuracy: 0.7029
Epoch 80/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 1.3613 - word_loss: 0.4778 - speaker_loss: 0.8835 - word_accuracy: 0.8486 - speaker_accuracy: 0.7536 - val_loss: 1.8941 - val_word_loss: 0.5853 - val_speaker_loss: 1.3389 - val_word_accuracy: 0.8335 - val_speaker_accuracy: 0.7061
Epoch 81/1000
84663/84663 [==============================] - 7s 77us/sample - loss: 1.3484 - word_loss: 0.4729 - speaker_loss: 0.8756 - word_accuracy: 0.8505 - speaker_accuracy: 0.7541 - val_loss: 1.8869 - val_word_loss: 0.5862 - val_speaker_loss: 1.3299 - val_word_accuracy: 0.8328 - val_speaker_accuracy: 0.7067
Epoch 82/1000
84663/84663 [==============================] - 6s 76us/sam

84663/84663 [==============================] - 6s 77us/sample - loss: 1.0761 - word_loss: 0.3937 - speaker_loss: 0.6823 - word_accuracy: 0.8749 - speaker_accuracy: 0.8009 - val_loss: 1.8122 - val_word_loss: 0.5848 - val_speaker_loss: 1.2603 - val_word_accuracy: 0.8422 - val_speaker_accuracy: 0.7286
Epoch 106/1000
84663/84663 [==============================] - 6s 75us/sample - loss: 1.0623 - word_loss: 0.3918 - speaker_loss: 0.6706 - word_accuracy: 0.8759 - speaker_accuracy: 0.8042 - val_loss: 1.8046 - val_word_loss: 0.5841 - val_speaker_loss: 1.2537 - val_word_accuracy: 0.8408 - val_speaker_accuracy: 0.7314
Epoch 107/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 1.0580 - word_loss: 0.3920 - speaker_loss: 0.6662 - word_accuracy: 0.8762 - speaker_accuracy: 0.8068 - val_loss: 1.8090 - val_word_loss: 0.5815 - val_speaker_loss: 1.2607 - val_word_accuracy: 0.8438 - val_speaker_accuracy: 0.7294
Epoch 108/1000
84663/84663 [==============================] - 6s 75us/

...

84663/84663 [==============================] - 6s 77us/sample - loss: 0.2093 - word_loss: 0.0792 - speaker_loss: 0.1301 - word_accuracy: 0.9752 - speaker_accuracy: 0.9574 - val_loss: 2.0773 - val_word_loss: 0.9280 - val_speaker_loss: 1.2074 - val_word_accuracy: 0.8628 - val_speaker_accuracy: 0.8089
Epoch 860/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 0.2142 - word_loss: 0.0808 - speaker_loss: 0.1334 - word_accuracy: 0.9749 - speaker_accuracy: 0.9559 - val_loss: 2.0843 - val_word_loss: 0.9256 - val_speaker_loss: 1.2144 - val_word_accuracy: 0.8636 - val_speaker_accuracy: 0.8080
Epoch 861/1000
84663/84663 [==============================] - 6s 77us/sample - loss: 0.2088 - word_loss: 0.0783 - speaker_loss: 0.1305 - word_accuracy: 0.9755 - speaker_accuracy: 0.9572 - val_loss: 2.0801 - val_word_loss: 0.9205 - val_speaker_loss: 1.2026 - val_word_accuracy: 0.8638 - val_speaker_accuracy: 0.8115
Epoch 862/1000
84663/84663 [==============================] - 6s 76us/

84663/84663 [==============================] - 6s 76us/sample - loss: 0.2018 - word_loss: 0.0721 - speaker_loss: 0.1298 - word_accuracy: 0.9768 - speaker_accuracy: 0.9576 - val_loss: 2.0845 - val_word_loss: 0.9199 - val_speaker_loss: 1.2085 - val_word_accuracy: 0.8627 - val_speaker_accuracy: 0.8089
Epoch 886/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 0.2004 - word_loss: 0.0731 - speaker_loss: 0.1273 - word_accuracy: 0.9768 - speaker_accuracy: 0.9594 - val_loss: 2.0891 - val_word_loss: 0.9306 - val_speaker_loss: 1.2050 - val_word_accuracy: 0.8629 - val_speaker_accuracy: 0.8101
Epoch 887/1000
84663/84663 [==============================] - 6s 77us/sample - loss: 0.2010 - word_loss: 0.0757 - speaker_loss: 0.1254 - word_accuracy: 0.9763 - speaker_accuracy: 0.9594 - val_loss: 2.0844 - val_word_loss: 0.9230 - val_speaker_loss: 1.2161 - val_word_accuracy: 0.8636 - val_speaker_accuracy: 0.8109
Epoch 888/1000
84663/84663 [==============================] - 6s 76us/

84663/84663 [==============================] - 6s 76us/sample - loss: 0.1996 - word_loss: 0.0754 - speaker_loss: 0.1243 - word_accuracy: 0.9763 - speaker_accuracy: 0.9594 - val_loss: 2.1110 - val_word_loss: 0.9222 - val_speaker_loss: 1.2245 - val_word_accuracy: 0.8633 - val_speaker_accuracy: 0.8109
Epoch 912/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 0.2074 - word_loss: 0.0749 - speaker_loss: 0.1324 - word_accuracy: 0.9763 - speaker_accuracy: 0.9579 - val_loss: 2.0793 - val_word_loss: 0.9139 - val_speaker_loss: 1.2134 - val_word_accuracy: 0.8636 - val_speaker_accuracy: 0.8118
Epoch 913/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 0.2037 - word_loss: 0.0748 - speaker_loss: 0.1289 - word_accuracy: 0.9764 - speaker_accuracy: 0.9580 - val_loss: 2.0889 - val_word_loss: 0.9284 - val_speaker_loss: 1.2092 - val_word_accuracy: 0.8627 - val_speaker_accuracy: 0.8095
Epoch 914/1000
84663/84663 [==============================] - 6s 76us/

84663/84663 [==============================] - 6s 76us/sample - loss: 0.1935 - word_loss: 0.0710 - speaker_loss: 0.1225 - word_accuracy: 0.9779 - speaker_accuracy: 0.9598 - val_loss: 2.0832 - val_word_loss: 0.9369 - val_speaker_loss: 1.1966 - val_word_accuracy: 0.8633 - val_speaker_accuracy: 0.8141
Epoch 938/1000
84663/84663 [==============================] - 6s 75us/sample - loss: 0.1987 - word_loss: 0.0736 - speaker_loss: 0.1251 - word_accuracy: 0.9771 - speaker_accuracy: 0.9590 - val_loss: 2.0704 - val_word_loss: 0.9274 - val_speaker_loss: 1.2026 - val_word_accuracy: 0.8621 - val_speaker_accuracy: 0.8109
Epoch 939/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 0.1988 - word_loss: 0.0728 - speaker_loss: 0.1260 - word_accuracy: 0.9775 - speaker_accuracy: 0.9588 - val_loss: 2.1046 - val_word_loss: 0.9409 - val_speaker_loss: 1.2215 - val_word_accuracy: 0.8620 - val_speaker_accuracy: 0.8088
Epoch 940/1000
84663/84663 [==============================] - 6s 76us/

84663/84663 [==============================] - 7s 78us/sample - loss: 0.1905 - word_loss: 0.0683 - speaker_loss: 0.1223 - word_accuracy: 0.9786 - speaker_accuracy: 0.9598 - val_loss: 2.0825 - val_word_loss: 0.9479 - val_speaker_loss: 1.1900 - val_word_accuracy: 0.8631 - val_speaker_accuracy: 0.8147
Epoch 964/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 0.1894 - word_loss: 0.0682 - speaker_loss: 0.1213 - word_accuracy: 0.9785 - speaker_accuracy: 0.9602 - val_loss: 2.0988 - val_word_loss: 0.9509 - val_speaker_loss: 1.1974 - val_word_accuracy: 0.8634 - val_speaker_accuracy: 0.8120
Epoch 965/1000
84663/84663 [==============================] - 6s 76us/sample - loss: 0.1967 - word_loss: 0.0724 - speaker_loss: 0.1244 - word_accuracy: 0.9778 - speaker_accuracy: 0.9590 - val_loss: 2.1101 - val_word_loss: 0.9544 - val_speaker_loss: 1.2058 - val_word_accuracy: 0.8635 - val_speaker_accuracy: 0.8142
Epoch 966/1000
84663/84663 [==============================] - 6s 76us/

In [8]:
print(trainX[5:6,:,:].shape)
predWord,predSpeaker=model.predict(trainX[5:6,:,:])
#print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_.shape)
print(ea.leSpeakers.classes_.shape)

(1, 30, 20)
12 = 1.0
593 = 0.99169403
52e228e9  ha dicho ' marvin '
(30,)
(1881,)


In [128]:
!sox -r 16000 -t alsa default recording.wav silence 1 0.1 1% 1 1.5 1%
!aplay "recording.wav"


Input File     : 'default' (alsa)
Channels       : 2
Sample Rate    : 16000
Precision      : 16-bit
Sample Encoding: 16-bit Signed Integer PCM

In:0.00% 00:00:02.30 [00:00:00.00] Out:3.55k [      |      ]        Clip:0    
Done.
Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [129]:
!aplay "recording.wav"

Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [130]:
sample=wav2mfcc("recording.wav")
sample=np.expand_dims(np.array(sample),axis=0)
sample.shape

(1, 30, 20)

In [131]:
predWord,predSpeaker=model.predict([sample])
#print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_.shape)

8 = 0.99951184
211 = 0.9269406
1df99a8a  ha dicho ' go '
['bed' 'bird' 'cat' 'dog' 'down' 'eight' 'five' 'four' 'go' 'happy'
 'house' 'left' 'marvin' 'nine' 'no' 'off' 'on' 'one' 'right' 'seven'
 'sheila' 'six' 'stop' 'three' 'tree' 'two' 'up' 'wow' 'yes' 'zero']
(1881,)


In [7]:
model.save(os.path.join('./','tf2_keras_asr_google_commands_v0.02_20061101.h5'))    # guarda el modelo